# Airportia.com Flight Traffic Webscraper

This selenium-driven HTML webscraper plots total flights to or from a specified airport for a specified date range by origin or destination depending on whether the user chooses to investigate arrivals or departures. The unfortunate limitation here is that airportia.com only carries flight data for a week before dropping it from accessibility, thus the scraper would need to be run weekly in order to put together a long-term dataset. 

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

import json

import datetime as dt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML, display
from ipywidgets import Dropdown

import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
# from selenium.webdriver.support.expected_conditions import visibility_of_element_located

In [2]:
chromeOptions = webdriver.ChromeOptions() 
chromeOptions.add_argument("enable-automation")
chromeOptions.add_argument("--disable-gpu")
chromeOptions.add_argument("--start-maximized")
chromeOptions.add_argument("--no-sandbox")
chromeOptions.add_argument("--disable-extensions")
chromeOptions.add_argument("--dns-prefetch-disable")
# chromeOptions.add_argument("--window-size=1920,1080")
# chromeOptions.add_argument("--headless")

# chromeOptions.add_argument("--proxy-server='direct://'")
# chromeOptions.add_argument("--proxy-bypass-list=*")
# chromeOptions.add_argument("--disable-dev-shm-usage")
# chromeOptions.add_argument("--ignore-certificate-errors")
# chromeOptions.add_experimental_option("excludeSwitches", ["enable-automation"])
# chromeOptions.add_experimental_option('useAutomationExtension', False)

PATH = 'C:\Program Files (x86)\chromedriver.exe'

In [3]:
# builds and saves a nested dictionary with Countries as keys, and key/value pair containing airports and links for their flight traffic info as values 

# driver = webdriver.Chrome(PATH,options=chromeOptions)
# driver.get('https://www.airportia.com/airports/')

# size = len(driver.find_elements_by_xpath('/html/body/div[2]/div[1]/div[6]/div/a'))
# country_dict = {}
# for i in range(1,size+1):
#     path = "/html/body/div[2]/div[1]/div[6]/div/a[{num}]".format(num=i)
#     data = driver.find_element_by_xpath(path).text
#     data2 = driver.find_element_by_xpath(path).get_attribute("href")
#     country_dict[data] = data2    
    
# driver.quit()

# country_dict2 = {}

# for country, link in country_dict.items():
#     driver = webdriver.Chrome(PATH,options=chromeOptions)
#     driver.get(link)
    
#     airport_dict = {}
#     size = len(driver.find_elements_by_xpath('/html/body/div[2]/div[1]/div[3]/div/a'))
    
#     for i in range(1,size+1):
#         path2 = "/html/body/div[2]/div[1]/div[3]/div/a[{num}]".format(num=i)
#         data = driver.find_element_by_xpath(path2).text
#         data2 = driver.find_element_by_xpath(path2).get_attribute("href")
#         airport_dict[data] = data2
    
#     driver.quit()
    
#     country_dict2[country] = airport_dict    
    
# with open('cal', 'w') as fp:
#     json.dump(country_dict2, fp)

In [4]:
with open('cal','r') as fp:
    cal = json.load(fp)

In [6]:
def airport_linker():
    
    country = countryW.value
    airport = airportW.value
    aod = aodW.value
    
    airport_link = cal[country][airport]
    
    if aod == 'arrivals': 
        airport_link = airport_link + 'arrivals/'
    elif aod == 'departures': 
        airport_link = airport_link + 'departures/'   
        
    driver = webdriver.Chrome(PATH,options=chromeOptions)
    driver.get(airport_link)
    
    datevalues = []
    for i in range(1,7): 
        date = dt.datetime.today() - dt.timedelta(days=i)
        datevalues.append(date.strftime('%Y%m%d'))
          
    dfs = {}
    for date in datevalues: 
        select1 = Select(driver.find_element_by_id('airport_{}_date_range_date'.format(aod)))
        select1.select_by_value(date)
        select2 = Select(driver.find_element_by_id('airport_{}_date_range_from_time'.format(aod)))
        select2.select_by_value('0')
        select3 = Select(driver.find_element_by_id('airport_{}_date_range_to_time'.format(aod)))
        select3.select_by_value('23')
        select4 = driver.find_element_by_id('airport_{}_go'.format(aod))
        select5 = driver.find_element_by_xpath('//*[@id="airport_{}_go"]'.format(aod))
        driver.execute_script("arguments[0].click();", select5)    
    #     select5.click()
        list_of_flights2 = []
        
        size = len(driver.find_elements_by_xpath('/html/body/div[2]/div[1]/div[2]/div[2]/table/tbody/tr'))
        
        for i in range(2,size+1):
            row = []
            for x in range(1,7):
                path2 = "/html/body/div[2]/div[1]/div[2]/div[2]/table/tbody/tr[{num}]/td[{num2}]".format(num=i, num2=x)
                try:
                    dater = driver.find_element_by_xpath(path2).text
                except:
                    pass     
                row.append(dater)
            list_of_flights2.append(row)
#         nan_value = float("NaN")
        
        if aod == 'arrivals': 
            dfs[date] = pd.DataFrame(list_of_flights2,columns=['Flight','From','Airline','Scheduled','Arrival','Status'])
            dfs[date]['Date'] = date
        elif aod == 'departures': 
            dfs[date] = pd.DataFrame(list_of_flights2,columns=['Flight','To','Airline','Scheduled','Departure','Status'])
            dfs[date]['Date'] = date
        
    driver.quit()
    nan_value = float("NaN")    
    flights = pd.concat(dfs.values(),ignore_index=True)
    flights['Flight'].replace("", nan_value, inplace=True)
    flights.dropna(subset = ["Flight"], inplace=True)
    flights.reset_index(drop=True, inplace=True)
    flights = flights[(flights['Status'] != 'CANCELLED') & (flights['Status'] != 'Unknown')]
    flights = flights.drop_duplicates()
    flights.reset_index(drop=True,inplace=True)
   
    if aod == 'arrivals': 
        col = 'From'
    else: 
        col = 'To'
    
    flights_df = flights.drop(flights.columns[0],axis=1)
    flights_df = pd.DataFrame(flights_df.groupby(col)['Airline'].value_counts()).unstack()
    flights_df['sum'] = flights_df.sum(axis=1)
    flights_df = flights_df.sort_values('sum',ascending=False)
    flights_df.drop('sum',axis=1,inplace=True)
    flights_df = flights_df.head(34)
    flights_df = flights_df.dropna(axis=1,how='all')
    flights_df = flights_df['Airline'].copy()
    ax = flights_df.plot.barh(figsize=(15,15), colormap='tab20', xticks=range(0,60,5),grid=True,fontsize=20,stacked=True,width=.9)
    ax.set_xlabel('Number of Flights',fontsize=20)
    ax.set_ylabel('Destination',fontsize=20)
    plt.title('{Airport} {Way} {Start} - {End} (Total: {Total})'.format(Airport=airport, Way=aod, Start=datevalues[-1], End=datevalues[0], Total=len(flights)),fontsize=25)
    plt.legend(flights_df,fontsize=20)
    ax.invert_yaxis()
    plt.show()
    
#     return flights
#     print(flights)

def selections(country,airport,aod): 
    airportW.options = cal[countryW.value].keys()

countryW = widgets.Dropdown(options=cal.keys())
airportW = widgets.Dropdown(options=cal[countryW.value].keys())
aodW = widgets.Dropdown(options=['arrivals','departures'],description='Arrivals or Departures')
uno = widgets.interactive(selections, country=countryW, airport=airportW, aod=aodW)
buttoner = interact_manual.options(manual_name="Show Flights")
buttoner(airport_linker)

display(uno)   
display(buttoner)

interactive(children=(Button(description='Show Flights', style=ButtonStyle()), Output()), _dom_classes=('widge…

interactive(children=(Dropdown(description='country', options=('Afghanistan', 'Albania', 'Algeria', 'American …